# Calculator for common DSA-2000 Questions

## Casey Law, claw@astro.caltech.edu

Written in support of DSA-2000 science workshop (January 2022). Reproduces work by Steve Myers for VLASS and DSA-2000 survey design.

In [1]:
%matplotlib inline

import pylab as pl
import numpy as np
import healpy

## Define some useful functions


In [2]:
resolution = lambda freq, bmax: 1.22*3600*np.degrees(3e8/float(freq)/bmax) # resolution in asec for freq in Hz, bmax in meters
sensitivity = lambda sefd, dt, bw, eta, nbl: sefd/(eta*np.sqrt(nbl*2 * dt * bw * 2))    # sefd in Jy, dt in s, bw in Hz, assumes std correlator eficiency and 2 pols
tobs = lambda s0, sefd, na, bw, eta: (sefd/(s0*eta))**2/(na*(na-1)*bw*2)    # obsering time in s (inverse of sensitivity eqn)
area_field = lambda fov: 0.5665 * fov**2  # fov in amin
surveyspeed = lambda fov, t0: area_field(fov)/t0         # ss in deg2/hr, fov in amin, t0 is time per pointing in s
f_beam = lambda rad, fwhm: np.exp(-4*np.log(2)*(rad/fwhm)**2)  # beam function (fractional power at radius rad)

datarate_vis = lambda na, nch, npol, tsamp: (na*(na-1)/2)*nch*npol*8/tsamp/1024**2  # MB/s for tsamp in s
datarate_vis_vla = lambda tsamp, nspw: 45*(nspw*64*4/16384.) / tsamp # scaling taken from nrao oss, minimal nchan, full pol

In [3]:
ess_nu = lambda fov, tobs, nu, alpha: surveyspeed(fov, tobs) * nu**(2*alpha)
ess_flux = lambda fov, tobs, sensitivity, gamma: surveyspeed(fov, tobs) * sensitivity**(gamma+2)
ess_nuflux = lambda fov, tobs, nu, alpha, sensitivity, gamma: surveyspeed(fov, tobs) * sensitivity**(gamma+2) * nu**(2*alpha)
surveytime = lambda band, sensitivity, area, overhead: overhead * area/surveyspeed(fov[band], tobs(sensitivity, sefd[band], na[band], bw_max[band], eta[band]))   # total time in hours for given band, area in sq deg and sensitivity in Jy

# Effective survey speed for extragalactic specs (alpha = -0.7, gamma => euclidean).
ess_eg = lambda fov, dt, nu, sensitivity: ess_nuflux(fov, dt, nu, -0.7, sensitivity, -1.5)
sigma_confusion = lambda nu, theta: 1.2e-6*(nu/3.02e9)**(-0.7)*(theta/8)**(10/3)  # confusion with freq in Hz, theta in asec (Condon et al 2012)
survey_area = lambda dec_lim: (1+np.sin(np.radians(np.abs(dec_lim))))*(2*np.pi*(180/np.pi)**2)  # sq deg, exclude outside dec_lim

In [4]:
def find_hpnside(size, verbose=False):
    """ Use healpix scheme to find optimal nsize for a given size in degrees
    """
    for i in range(20):
        resol = np.degrees(healpy.nside2resol(2**i))
        if resol < size:
            if verbose:
                print(f'nside=2**{i} has resolution of {resol:1.1e} deg (resolves limit of {size} deg)')
            break
    return 2**i

In [5]:
# cumulative histogram of baseline lengths for W configuration
# nblarr[0] is number of baselines, nblarr[1] is max baseline length in meters.
blfrac = np.array([   2713.,    4985.,    9054.,   15119.,   22950.,   32325.,
          43181.,   55824.,   69858.,   85469.,  102308.,  120628.,
         140305.,  161417.,  183853.,  207582.,  231966.,  258083.,
         285248.,  313273.,  342334.,  372247.,  403392.,  435111.,
         468076.,  501321.,  535350.,  570064.,  605587.,  641260.,
         677628.,  714315.,  751568.,  788970.,  826299.,  863931.,
         901520.,  939318.,  977137., 1014510., 1051771., 1088905.,
        1126009., 1162462., 1198652., 1234562., 1269596., 1304725.,
        1338824., 1372722., 1405761., 1438268., 1469989., 1501154.,
        1531615., 1561165., 1589887., 1617913., 1645118., 1670979.,
        1696439., 1720885., 1744597., 1767487., 1789383., 1810425.,
        1830348., 1849484., 1867862., 1885488., 1902348., 1918459.,
        1933525., 1947866., 1961529., 1974579., 1986612., 1998076.,
        2009133., 2019197., 2028535., 2037223., 2045349., 2052889.,
        2059729., 2065997., 2071635., 2076675., 2081213., 2085130.,
        2088597., 2091336., 2093612., 2095354., 2096617., 2097443.,
        2097907., 2098089., 2098162., 2098176.])/2098176
minbl = np.array([    153.93621821,   307.87243643,   461.80865464,
          615.74487286,   769.68109107,   923.61730929,  1077.5535275 ,
         1231.48974571,  1385.42596393,  1539.36218214,  1693.29840036,
         1847.23461857,  2001.17083679,  2155.107055  ,  2309.04327322,
         2462.97949143,  2616.91570964,  2770.85192786,  2924.78814607,
         3078.72436429,  3232.6605825 ,  3386.59680072,  3540.53301893,
         3694.46923714,  3848.40545536,  4002.34167357,  4156.27789179,
         4310.21411   ,  4464.15032822,  4618.08654643,  4772.02276464,
         4925.95898286,  5079.89520107,  5233.83141929,  5387.7676375 ,
         5541.70385572,  5695.64007393,  5849.57629215,  6003.51251036,
         6157.44872857,  6311.38494679,  6465.321165  ,  6619.25738322,
         6773.19360143,  6927.12981965,  7081.06603786,  7235.00225607,
         7388.93847429,  7542.8746925 ,  7696.81091072,  7850.74712893,
         8004.68334715,  8158.61956536,  8312.55578357,  8466.49200179,
         8620.42822   ,  8774.36443822,  8928.30065643,  9082.23687465,
         9236.17309286,  9390.10931108,  9544.04552929,  9697.9817475 ,
         9851.91796572, 10005.85418393, 10159.79040215, 10313.72662036,
        10467.66283858, 10621.59905679, 10775.535275  , 10929.47149322,
        11083.40771143, 11237.34392965, 11391.28014786, 11545.21636608,
        11699.15258429, 11853.08880251, 12007.02502072, 12160.96123893,
        12314.89745715, 12468.83367536, 12622.76989358, 12776.70611179,
        12930.64233001, 13084.57854822, 13238.51476643, 13392.45098465,
        13546.38720286, 13700.32342108, 13854.25963929, 14008.19585751,
        14162.13207572, 14316.06829393, 14470.00451215, 14623.94073036,
        14777.87694858, 14931.81316679, 15085.74938501, 15239.68560322,
        15393.62182144])


def frac(scale):
    """ Fraction of cumulative distribution seen by scale in arcmin
    """

    km = 1.22*60*np.degrees((3e8/1.35e9))/scale # arcmin to km
    if km > max(minbl):
        i = len(minbl)-1
    elif km < min(minbl):
        return 0.
    else:
        i = np.where(minbl > km)[0][0]
    return blfrac[i]


## Set telescope parameters as a function of band/config

In [6]:
# keys defined for configurations and bands separately.
names = ['DSA', 'VLA', 'ASKAP']
na = {'DSA': 2000, 'VLA': 27, 'ASKAP': 36}   # number of antennas
freq = {'DSA': 1.35e9, 'VLA': 3.0e9, 'ASKAP': 1.28e9}  # band center
fov = {'DSA': 60*3.06, 'VLA': 15., 'ASKAP': 60*3.}    # fwhm field of view (theta_pb) in arcminutes
bw_max = {'DSA': 0.65*1.3e9, 'VLA': 1.5e9, 'ASKAP': 288e6}    # bandwidth (minus RFI)
bmax = {'DSA': 17e3, 'VLAB': 11.1e3, 'ASKAP': 6e3}    # longest baselines in meter
sefd = {'DSA': 5020, 'VLA': 370, 'ASKAP': 1952}    # array SEFD
# VLA sefd values appear high, because we assume 27 ants, whereas exposure calculator uses 25 ants. result is the same in both cases.
eta = {'DSA': 1.0, 'VLA': 0.92, 'ASKAP': 1.0}   # correlator efficiency
surveyspecs = {'DSA All-Sky': [3e4, 2.04e-6, 1.25, 1.0],
               'VLASS': [3.3885e4, 122e-6, 1.17, 1.09],
               'ASKAP EMU': [3.0939e4, 10e-6, 1.25, 1.0]}  # something must be wrong with ASKAP numbers

# Q&A

### Given sensitivity/area, what is the cadence?

In [7]:
s0 = 2e-6
a0 = 3e4
o0 = 1.25
t_total = 5 * 365.24 * 24  # 5 years in hrs
frac_survey = 0.65
t_epoch = surveytime('DSA', s0, a0, o0)
print(f'{t_epoch:.0f} hrs to cover {a0:.0f} sq deg to depth of {s0:.1e} Jy with {frac_survey*t_total/t_epoch:.1f} epochs in {t_total/(24*365.24):.1f} years')

1831 hrs to cover 30000 sq deg to depth of 2.0e-06 Jy with 15.6 epochs in 5.0 years


### And for deep field?

In [8]:
s0 = 100e-9
a0 = 30.
o0 = 1.25
t_total = 5 * 365.24 * 24  # 5 years in hrs
frac_deep = 0.05
t_survey = surveytime('DSA', s0, a0, o0)
print(f'{t_survey:.0f} hrs to cover {a0:.0f} sq deg to depth of {s0:.1e} Jy with {frac_deep*t_total/t_epoch:.1f} epochs in {t_total/(24*365.24):.1f} years')

732 hrs to cover 30 sq deg to depth of 1.0e-07 Jy with 1.2 epochs in 5.0 years


### Given cadence/area, what is the sensitivity?

In [9]:
band = 'DSA'
a0 = 3e4
n_visits = 16
cadence = n_visits/5*365.24*24 # per hour
o0 = 1.25

n_fields = a0/area_field(fov[band]/60)
t_field = cadence/n_fields

print(f'{a0} deg^2 covered with {n_fields:.0f} fields of {area_field(fov[band]/60):.1f} deg^2 to depth {sensitivity(sefd[band], t_field/o0, bw_max[band], eta[band], (na[band]*(na[band]-1)/2)):.1e} Jy for each of {n_visits} epoch')

30000.0 deg^2 covered with 5656 fields of 5.3 deg^2 to depth 3.1e-05 Jy for each of 16 epoch


### How long until reaching the confusion limit?

In [10]:
theta0 = resolution(freq['DSA'], bmax['DSA'])
sigma_c0 = sigma_confusion(freq['DSA'], theta0)
t0 = tobs(sigma_c0, sefd['DSA'], na['DSA'], bw_max['DSA'], eta['DSA'])

print(f"Confusion limit of {1e9*sigma_c0:.0f} nJy reached in {t0/3600:.1f} hr ({theta0:0.1f} arcsec beam at {freq['DSA']/1e9} GHz)")

Confusion limit of 109 nJy reached in 87.2 hr (3.3 arcsec beam at 1.35 GHz)


### Which deep fields are best?

In [11]:
# TBD: time to cover existing deep fields
field_properties = {'COSMOS': [], 'HDFS': [], 'HUDF': [], 'CDFS': []}

### How many radio continuum sources detected?

In [48]:
# relevant papers: Condon et al 2012, Matthews et al 2020

s0 = 1e-1
a0 = 4e4

#n_S = lambda s, a: (9000*s**(-1.7)) * a*(np.pi/180)**2
#n_S = lambda s, a: np.exp(2.718 + 0.405*(np.log(s) + 5) - 0.02*(np.log(s) + 5)**2 + 0.019*(np.log(s) + 5)**3)
n_S = lambda s, a: 1000 * np.exp((-4*np.log(2)/2**2)*(np.log(s) - np.log(1e-1))**2)

print(f'n = {n_S(s0, a0):.1e} sources')
#n_source = lambda s_lim, resolution: ...

n = 1.0e+03 sources


### What is the sensitivity to extended sources?

In [32]:
dt = 900  # seconds
scale = 1  # arcmin
nbl = (na['DSA']*(na['DSA']-1)/2)*frac(scale)
s0 = sensitivity(sefd['DSA'], dt, bw_max['DSA'], eta['DSA'], nbl)
print(f'On scale of {scale} arcmin, sensitivity of {s0} Jy in {dt} seconds')

On scale of 1 arcmin, sensitivity of 1.4190324435893207e-05 Jy in 900 seconds


## What physical scales are accesible to HI image

In [33]:
# consider D=100 Mpc, z=0.1, z=1.0
# HI column and velocity resolution

### What sensitivity to Galactic HI/OH?

In [34]:
# calc column density, velocity range, velocity resolution
# reference value 40 microJy/beam/ch(134kHz), 3.3" beam
sensitivity(sefd['DSA'], 900, 134e3, eta['DSA'], na['DSA']*(na['DSA']-1)/2)

0.00016165668566682666

### What minimum declination is best for all-sky survey?

In [35]:
dec_min = -30
area30 = survey_area(dec_min)

# For elliptical array (17 km N-S vs 15 km E-W baselines), calc southerly zenith angle for circular beam
za0 = np.degrees(np.cos(15/17))
lat = +39  # rough latitude of array
beam_elong = np.cos(np.radians((lat-za0)-dec_min))
print(f'At min Dec of {dec_min}, the survey area is {area30:.0f} and transit beam elongation is {beam_elong:0.2f}')

At min Dec of -30, the survey area is 30940 and transit beam elongation is 0.84


### How many pointings to cover the sky?

In [36]:
dec_min = -30
nside_thetap = find_hpnside(fov['DSA']/60)
n_pointings = int(np.round(healpy.nside2npix(nside_thetap)*(survey_area(dec_min)/survey_area(-90))))
thetap_area = healpy.nside2pixarea(nside_thetap, degrees=True)
print(f'Cover the sky with {n_pointings} pointings of healpix area {thetap_area:.2f} deg^2')

Cover the sky with 9216 pointings of healpix area 3.36 deg^2


### What is relative sensitivity at top/bottom of band?

### How big is all-sky image?

In [37]:
oversample = 4  # pixel/beam
nside_thetab = find_hpnside(resolution(freq['DSA'], bmax['DSA'])/3600/oversample)
n_pix_tot = int(np.round(healpy.nside2npix(nside_thetab)*(survey_area(dec_min)/survey_area(-90))))
thetab_resol = healpy.nside2resol(nside_thetab, arcmin=True)*60
print(f'Pixelate the sky with {n_pix_tot/1024**4} Tpixels of resolution {thetab_resol:.2f} arcsec (healpix nside={nside_thetab})')
print(f'\tThis requires {8*n_pix_tot/1024**4} TB (Stokes I, one band)')

print(f'{nside_thetab//nside_thetap} pixels per pointing (thetab side length)')
print(f'{int(2*1.19*nside_thetab//nside_thetap)} pixels per pointing (first null side length)')

Pixelate the sky with 0.5625 Tpixels of resolution 0.81 arcsec (healpix nside=262144)
	This requires 4.5 TB (Stokes I, one band)
8192 pixels per pointing (thetab side length)
19496 pixels per pointing (first null side length)


# More General Comparisons

## Telescope performance

In [39]:
# something wrong with how I'm using ASKAP numbers, so skipping that for now

print(f'Array \t   Tobs (s) \t    SS (OTF; deg2/hr) \t Sens. (Jy) \t hrs/3e4 deg2 (ideal)')
print('----------------------------------------------------------------------------------------------')
for name in names:
    na0 = na[name]
    try:
        surveykey = list(filter(lambda x: name in x, surveyspecs.keys()))[0]
    except IndexError:
        continue
    s0 = surveyspecs[surveykey][1]
    t0 = tobs(s0, sefd[name], na0, bw_max[name], eta[name])
    s0 = sensitivity(sefd[name], t0, bw_max[name], eta[name], na0*(na0-1)/2)
    print(f'{name} \t   {t0:.1f}  \t\t {surveyspeed(fov[name], t0):.1f}   \t {s0:1.2e} \t {surveytime(name, s0, 30000, 1.0):.1f} ')

Array 	   Tobs (s) 	    SS (OTF; deg2/hr) 	 Sens. (Jy) 	 hrs/3e4 deg2 (ideal)
----------------------------------------------------------------------------------------------
DSA 	   896.2  		 21.3   	 2.04e-06 	 1408.0 
VLA 	   5.2  		 24.7   	 1.22e-04 	 1214.5 
ASKAP 	   52500.9  		 0.3   	 1.00e-05 	 85811.0 


## Time required for surveys

In [40]:
def print_survey(surveyspecs):
    ss = 'Survey    \t s_lim   \t SS     \t Time/pointing \t Total Time\n'
    ss += '          \t (Jy)    \t (deg/hr)  \t (s)    \t   (hr)\n'
#    ss +='--------------------------------------------------------------------\n'
    for name in names:
        surveykey = list(filter(lambda x: name in x, surveyspecs.keys()))[0]
        specs = surveyspecs[surveykey]
        a0, s0, o0, tsysfac = specs
        na0 = na[name]
        tint = tobs(s0, sefd[name], na0, bw_max[name], eta[name])
        t0_pointing = tobs(s0, sefd[name], na0, bw_max[name], eta[name])
        t0_survey = tsysfac*surveytime(name, s0, a0, o0)
        surveyspeed0 = surveyspeed(fov[name], t0_pointing)
        ss += f'{surveykey}    \t {s0:.2e} \t {surveyspeed0:.1f}    \t {t0_pointing:.1f}    \t {t0_survey:.1f}\n'
    print(ss)

print_survey(surveyspecs)

Survey    	 s_lim   	 SS     	 Time/pointing 	 Total Time
          	 (Jy)    	 (deg/hr)  	 (s)    	   (hr)
DSA All-Sky    	 2.04e-06 	 21.3    	 896.2    	 1760.0
VLASS    	 1.22e-04 	 24.7    	 5.2    	 1749.4
ASKAP EMU    	 1.00e-05 	 0.3    	 52500.9    	 110621.1



## Other DSA-2000 surveys/products

- Spectroscopy
-- 9700 channels at 134kHz resolution
-- 4700 channels at 8.4kHz (HI/OH)
- Pulsars/FRBs
-- 16384 channels, 0.8 ms sampling with 20 seconds triggers => 3 GB/beam/trigger
-- 1 s dedispersed voltage cutout per candidate, 2e9 samples/s => 64 TB/trigger 
- Polarimetry
-- 1000 channels (1.3 MHz, Stokes QUV)